In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
from __future__ import unicode_literals
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# Run in python console
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jaimemillan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words(['english','danish', 'dutch', 'english', 'finnish', 'french', 'german', 'hungarian', 'italian', 'norwegian', 'portuguese', 'russian', 'spanish', 'swedish', 'turkish'])
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [4]:
import string 
strings = string.ascii_lowercase[1:26]
print(strings)

bcdefghijklmnopqrstuvwxyz


In [5]:
#new_df.columns = ['a', 'b','c','d','e','f','g','h','i','j','k','l']

In [19]:
df = pd.read_csv("a.csv",header=None)
#new_df = pd.read_csv('J_tsar_tweets.csv')

In [20]:
df.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11], dtype='int64')

In [21]:
nbr_data =200

In [22]:
new_df = pd.DataFrame({'content':df.iloc[:,10]})

In [23]:
bad_ids= []
data = new_df.content[2:nbr_data].values.tolist()
#data = new_df.text[2:nbr_data].values.tolist()
new_data = []
for idx,_sentence in enumerate(data):
    
    if isinstance(_sentence,float):
        bad_ids.append(idx)
        continue
        
    tokens = _sentence.split()
    new_string = str()
    
    ADD_STRING = True
    for t in tokens:
        try:
            t.encode('utf-8')
            
            new_string = new_string +  t.encode('utf-8')+ ' ' 
        except:
            bad_ids.append(idx)
            ADD_STRING = False
            break
    if ADD_STRING:
        new_data.append(new_string)
   
    

In [24]:

data = list(new_data)
len(data)

129

In [25]:
# Convert to list

#data = new_df.content[:50].values.tolist()


# Remove Emails
new_data = []
for idx,sentence in enumerate(data):
    new_sentence=str()
    try:
        
        new_sentence =  re.sub('\S*@\S*\s?', '', sentence)
        
    except:
         continue;
            
    try:
        
        new_sentence =  re.sub('\s+', ' ', new_sentence)
        
    except:
         continue;   
    
    try:
        
        new_sentence =  re.sub("\'", "", new_sentence)
        
    except:
         continue;

    new_data.append(new_sentence)
        
# Remove new line characters
#data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
#data = [re.sub("\'", "", sent) for sent in data]

#pprint(new_data[:])

In [26]:
data = new_data
len(data)

129

In [27]:
def sent_to_words(sentences):
    for sentence in sentences:
        
        try:
            a = gensim.utils.simple_preprocess(str(sentence), deacc=True)
            
            yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        except:
            
            continue
        
data_words = list(sent_to_words(data))

print(data_words[:2])
del data_words[0]

[[u'we', u'chat', u'about', u'the', u'movies', u'tv', u'and', u'pop', u'culturey', u'stuff'], [u'florence', u'augusta', u'merriam', u'bailey', u'was', u'an', u'american', u'ornithologist', u'and', u'nature', u'writer', u'she', u'started', u'observing', u'bird', u'behavior', u'at', u'time', u'when', u'most', u'bird', u'study', u'was', u'based', u'on', u'collections', u'and', u'skins', u'by', u'she', u'began', u'to', u'write', u'articles', u'focusing', u'on', u'protecting', u'birds', u'her', u'introduction', u'of', u'birdwatching', u'field', u'guide', u'aimed', u'at', u'living', u'birds', u'is', u'considered', u'the', u'first', u'in', u'the', u'tradition', u'of', u'modern', u'bird', u'guides', u'she', u'wrote', u'the', u'first', u'of', u'these', u'at', u'the', u'age', u'of', u'initially', u'as', u'series', u'of', u'notes', u'in', u'the', u'audubon', u'magazine', u'and', u'later', u'as', u'books', u'in', u'birding', u'on', u'bronco', u'she', u'writes', u'an', u'engaging', u'memoir', u'abo

In [28]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/jaimemillan/miniconda2/lib/python2.7/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


[u'florence', u'augusta', u'merriam', u'bailey', u'was', u'an', u'american', u'ornithologist', u'and', u'nature', u'writer', u'she', u'started', u'observing', u'bird', u'behavior', u'at', u'time', u'when', u'most', u'bird', u'study', u'was', u'based', u'on', u'collections', u'and', u'skins', u'by', u'she', u'began', u'to', u'write', u'articles', u'focusing', u'on', u'protecting', u'birds', u'her', u'introduction', u'of', u'birdwatching', u'field', u'guide', u'aimed', u'at', u'living', u'birds', u'is', u'considered', u'the', u'first', u'in', u'the', u'tradition', u'of', u'modern', u'bird', u'guides', u'she', u'wrote', u'the', u'first', u'of', u'these', u'at', u'the', u'age', u'of', u'initially', u'as', u'series', u'of', u'notes', u'in', u'the', u'audubon', u'magazine', u'and', u'later', u'as', u'books', u'in', u'birding', u'on', u'bronco', u'she', u'writes', u'an', u'engaging', u'memoir', u'about', u'her', u'several', u'trips', u'to', u'study', u'birds', u'on', u'ranch', u'in', u'califo

In [29]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [30]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:10])

[[u'florence', u'augusta', u'merriam', u'bailey', u'american', u'ornithologist', u'nature', u'writer', u'start', u'observe', u'bird', u'behavior', u'time', u'bird', u'study', u'base', u'collection', u'skin', u'begin', u'write', u'article', u'focus', u'protect', u'bird', u'introduction', u'birdwatching', u'field', u'guide', u'aim', u'live', u'bird', u'consider', u'first', u'tradition', u'modern', u'bird', u'guide', u'write', u'first', u'age', u'initially', u'series', u'note', u'magazine', u'later', u'book', u'bird', u'bronco', u'write', u'engage', u'memoir', u'several', u'trip', u'study', u'bird', u'ranch', u'california', u'late'], [u'radio', u'show', u'play', u'bluegrass', u'old', u'time', u'gospel', u'music', u'unsigned', u'band', u'discuss', u'play', u'style'], [u'blend', u'soul', u'pop', u'funk', u'rhythm', u'blue', u'rock', u'song', u'musilosophy', u'singer', u'songwriter', u'jazz', u'pianist', u'lt_br', u'gt'], [u'great', u'podcast', u'host', u'libsyn'], [u'semi', u'weekly', u'dis

In [ ]:
#Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [ ]:
id2word[1]

In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

In [ ]:
len(corpus)





In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=2, 
                                           random_state=400,
                                           update_every=100,
                                           chunksize=10,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

In [ ]:
N_TOPICS = 10
lsi_model = gensim.models.LsiModel(corpus=corpus, num_topics=N_TOPICS, id2word=id2word)

In [ ]:
# Compute Perplexity
#print('\nPerplexity: ', lsi_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=lsi_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
print("LSI Model:")
 
for idx in range(N_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lsi_model.print_topic(idx, 10))
 
print("=" * 20)

In [ ]:
N_TOPICS = 5
lda_model = gensim.models.LdaModel(corpus=corpus, num_topics=N_TOPICS, id2word=id2word)

In [ ]:
print("LDA Model:")
 
for idx in range(N_TOPICS):
    # Print the first 10 most representative topics
    print("Topic #%s:" % idx, lda_model.print_topic(idx, 10))
 
print("=" * 20)

In [ ]:
test_data = pd.read_csv('J_tsar_tweets.csv')
test_data.columns

In [11]:
text = test_data.text.values

NameError: name 'test_data' is not defined

In [12]:
# Convert to list

#data = new_df.content[:50].values.tolist()


# Remove Emails
new_data = []
for idx,sentence in enumerate(text):
    new_sentence=str()
    try:
        
        new_sentence =  re.sub('\S*@\S*\s?', '', sentence)
        
    except:
         continue;
            
    try:
        
        new_sentence =  re.sub('\s+', ' ', new_sentence)
        
    except:
         continue;   
    
    try:
        
        new_sentence =  re.sub("\'", "", new_sentence)
        
    except:
         continue;

    new_data.append(new_sentence)

NameError: name 'text' is not defined

In [13]:
new_data[:2]

[]

In [14]:
string_text = list()
for tweet in new_data[:20]:
    tokens = tweet.split()
    
    string_text += tokens
    
#print(string_text)

In [15]:


bow = id2word.doc2bow(string_text)
 
print(lda_model[bow])
# [(0, 0.091615426138426506), (1, -0.0085557463300508351), (2, 0.016744863677828108), (3, 0.040508186718598529), (4, 0.014201267714185898), (5, -0.012208538275305329), (6, 0.031254053085582149), (7, 0.017529584659403553), (8, 0.056957633371540077),
#(9, 0.025989149894888153)]
 
#print(lda_model[bow])

NameError: name 'id2word' is not defined

In [16]:
print(lsi_model[bow])

NameError: name 'lsi_model' is not defined

In [17]:
top_words_per_topic = []
for t in range(lda_model.num_topics):
    top_words_per_topic.extend([(t, ) + x for x in lda_model.show_topic(t, topn = 20)])

pd.DataFrame(top_words_per_topic, columns=['Topic', 'Word', 'P']).to_csv("top_words.csv")

NameError: name 'lda_model' is not defined

In [18]:
top_words_lda = pd.read_csv('top_words.csv')

In [39]:
top_words_lda

,Unnamed: 0,Topic,Word,P
0,0,0,right,0.024787
1,1,0,series,0.022096
2,2,0,show,0.020418
3,3,0,quot,0.015878
4,4,0,episode,0.013829
5,5,0,hard,0.012986
6,6,0,christma,0.012983
7,7,0,keep,0.011998
8,8,0,audio,0.011799
9,9,0,get,0.011732
